In [2]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for your data.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run(debug=True)


In [25]:
from dash import Dash, dcc, html, Input, Output, callback
#import plotly.express as px
import pandas as pd

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Textarea(
            placeholder='Enter a value...',
            value='',
            style={'width':'100%'},
            id='input'
        ),
        html.Button('submit')
    ]),
    html.Div(id='output')
    
])

@callback(
    Output(component_id='output', component_property='children'),
    Input(component_id='input', component_property='value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'
    
if __name__ == '__main__':
    app.run(debug=True)

In [19]:
help(dcc.Textarea)

Help on class Textarea in module dash.dcc.Textarea:

class Textarea(dash.development.base_component.Component)
 |  Textarea(id=undefined, value=undefined, autoFocus=undefined, cols=undefined, disabled=undefined, form=undefined, maxLength=undefined, minLength=undefined, name=undefined, placeholder=undefined, readOnly=undefined, required=undefined, rows=undefined, wrap=undefined, accessKey=undefined, className=undefined, contentEditable=undefined, contextMenu=undefined, dir=undefined, draggable=undefined, hidden=undefined, lang=undefined, spellCheck=undefined, style=undefined, tabIndex=undefined, title=undefined, n_blur=undefined, n_blur_timestamp=undefined, n_clicks=undefined, n_clicks_timestamp=undefined, loading_state=undefined, persistence=undefined, persisted_props=undefined, persistence_type=undefined, **kwargs)
 |  
 |  A Textarea component.
 |  A basic HTML textarea for entering multiline text.
 |  
 |  Keyword arguments:
 |  
 |  - id (string; optional):
 |      The ID of this c

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

# Import your RAG system function
#from your_rag_module import rag_query

def rag_query(x):
    return x 
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("RAG System Interface"),
    dcc.Input(id="question-input", type="text", placeholder="Enter your question here...", style={'width': '100%'}),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='answer-output', style={'whiteSpace': 'pre-line', 'marginTop': '20px'})
])

# Define the callback to update the output
@app.callback(
    Output('answer-output', 'children'),
    Input('submit-button', 'n_clicks'),
    State('question-input', 'value')
)
def update_output(n_clicks, question):
    if n_clicks > 0 and question:
        answer = rag_query(question)
        return f"Question: {question}\n\nAnswer: {answer}"
    return "Ask a question and click Submit to get an answer."

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import sqlite3
from datetime import datetime

# Import your RAG system function
#from your_rag_module import rag_query

# Initialize the Dash app
app = dash.Dash(__name__)

def rag_query(x):
    return x 
    
# Set up SQLite database
def init_db():
    conn = sqlite3.connect('rag_interactions.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS interactions
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  timestamp TEXT,
                  question TEXT,
                  answer TEXT)''')
    conn.commit()
    conn.close()

def save_interaction(question, answer):
    conn = sqlite3.connect('rag_interactions.db')
    c = conn.cursor()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO interactions (timestamp, question, answer) VALUES (?, ?, ?)",
              (timestamp, question, answer))
    conn.commit()
    conn.close()

# Initialize the database
init_db()

# Define the layout
app.layout = html.Div([
    html.H1("RAG System Interface"),
    dcc.Input(id="question-input", type="text", placeholder="Enter your question here...", style={'width': '100%'}),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='answer-output', style={'whiteSpace': 'pre-line', 'marginTop': '20px'}),
    html.Div(id='db-status', style={'marginTop': '10px', 'color': 'green'})
])

# Define the callback to update the output and save to database
@app.callback(
    [Output('answer-output', 'children'),
     Output('db-status', 'children')],
    Input('submit-button', 'n_clicks'),
    State('question-input', 'value')
)
def update_output_and_save(n_clicks, question):
    if n_clicks > 0 and question:
        answer = rag_query(question)
        save_interaction(question, answer)
        return (
            f"Question: {question}\n\nAnswer: {answer}",
            "Interaction saved to database."
        )
    return "Ask a question and click Submit to get an answer.", ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [1]:
from rag_functions import rag

/home/jafet-dev/miniconda3/envs/transit-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
rag("quén me puede detener")

'En Ciudad de México, pueden detenerte los agentes de tránsito, quienes están autorizados para infraccionar. Estos agentes deben identificarte, indicarte la infracción cometida y mostrarte el artículo del Reglamento que la fundamenta. También pueden ser agentes de seguridad ciudadana, en caso de que se trate de infracciones más graves que requieran remitir al infractor al Juez Cívico.'

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import sqlite3
from datetime import datetime
from rag_functions import rag


# Initialize the Dash app
app = dash.Dash(__name__)
    
# Set up SQLite database
def init_db():
    conn = sqlite3.connect('rag_interactions.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS interactions
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  timestamp TEXT,
                  question TEXT,
                  answer TEXT)''')
    conn.commit()
    conn.close()

def save_interaction(question, answer):
    conn = sqlite3.connect('rag_interactions.db')
    c = conn.cursor()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO interactions (timestamp, question, answer) VALUES (?, ?, ?)",
              (timestamp, question, answer))
    conn.commit()
    conn.close()

# Initialize the database
init_db()

# Define the layout
app.layout = html.Div([
    html.H1("Sitema RAG sobre reglamento de Transito CDMX"),
    dcc.Input(id="question-input", type="text", placeholder="Hola, realiza tu pregunta dentro de este apartado...", style={'width': '100%'}),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='answer-output', style={'whiteSpace': 'pre-line', 'marginTop': '20px'}),
    html.Div(id='db-status', style={'marginTop': '10px', 'color': 'green'})
])

# Define the callback to update the output and save to database
@app.callback(
    [Output('answer-output', 'children'),
     Output('db-status', 'children')],
    Input('submit-button', 'n_clicks'),
    State('question-input', 'value')
)
def update_output_and_save(n_clicks, question):
    if n_clicks > 0 and question:
        answer = rag(question)
        save_interaction(question, answer)
        return (
            f"Question: {question}\n\nAnswer: {answer}",
            "Interaction saved to database."
        )
    return "Aquí aparecerá tu respuesta :)", ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)